# Data Analysys

In [2]:
import os
import csv 
import yaml
import pandas as pd
import numpy as np


In [3]:
with open('../config/filepath.yml', 'r') as f:
    path_config = yaml.safe_load(f)

dude_dir = os.path.join('..', path_config['data']['DUD-E'])
alphafold_dir = os.path.join('..', path_config['data']['alphafold'])
smiles_dir = os.path.join('..', path_config['data']['smiles'])
output_dir = os.path.join('..', path_config['data']['output'])
hist_dir = os.path.join('..', path_config['data']['hist'])
preprocessed_dir = os.path.join('..', path_config['data']['preprocessed'])
sample_dir = os.path.join('..', path_config['data']['samples'])

analyze SMILES 

In [5]:
smiles_csv_path = os.path.join('..',path_config['data']['train'], 'chembl_35_train.csv')

active_count = 0
decoy_count = 0
longest_smiles = ''
longest_smiles_len = 0
longest_smiles_id = ''

with open(smiles_csv_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        if len(row['canonical_smiles']) > len(longest_smiles):
            longest_smiles = row['canonical_smiles']
            longest_smiles_len = len(row['canonical_smiles'])
            longest_smiles_id = row['chembl_id']

# print(f'Active: {active_count}')
# print(f'Decoy: {decoy_count}')
# print(f'total: {active_count + decoy_count}')
print(f'longest smiles: {longest_smiles}')
print(f'longest smiles length: {longest_smiles_len}')
print(f'longest smiles id: {longest_smiles_id}')


longest smiles: O=S(=O)([O-])OC[C@H]1O[C@H](O[C@@H]2[C@H](O)[C@@H](O[C@@H]3[C@H](OS(=O)(=O)[O-])[C@@H](O[C@H]4[C@H](O)[C@@H](COS(=O)(=O)[O-])O[C@H](O[C@@H]5[C@H](O)[C@@H](O[C@H]6[C@H](O)[C@@H](COS(=O)(=O)[O-])O[C@H](O[C@@H]7[C@H](O)[C@@H](O[C@@H]8[C@H](OS(=O)(=O)[O-])[C@@H](O[C@H]9[C@H](O)[C@@H](COS(=O)(=O)[O-])O[C@H](O[C@@H]%10[C@H](O)[C@@H](O[C@H]%11[C@H](O)[C@@H](COS(=O)(=O)[O-])O[C@H](O[C@@H]%12[C@H](O)[C@@H](O[C@@H]%13[C@H](OS(=O)(=O)[O-])[C@@H](O)O[C@H](COS(=O)(=O)[O-])[C@H]%13OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]%12OS(=O)(=O)[O-])[C@H]%11OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]%10OS(=O)(=O)[O-])[C@H]9OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]8OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]7OS(=O)(=O)[O-])[C@H]6OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]5OS(=O)(=O)[O-])[C@H]4OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]3OS(=O)(=O)[O-])O[C@H](COS(=O)(=O)[O-])[C@H]2OS(=O)(=O)[O-])[C@@H](OS(=O)(=O)[O-])[C@@H](O[C@H]2O[C@H](COS(=O)(=O)[O-])[C@@H](OS(=O)(=O)[O-])[C@H](O[C@H]3O[C@H

analize smiles embedding

In [42]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-77M-MLM")
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-77M-MLM")  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def smiles_to_token_embeddings(smiles_list):
    """
    SMILES のトークンごとの埋め込みを取得する関数。

    Parameters:
    - smiles_list: list of str
        SMILES のリスト

    Returns:
    - token_embeddings: torch.Tensor
        各トークンの埋め込み (Shape: (batch_size, seq_len, hidden_size))
    - attention_mask: torch.Tensor
        アテンションマスク (Shape: (batch_size, seq_len))
    """
    # トークナイズ
    encoded_inputs = tokenizer(smiles_list, padding=True, truncation=True, return_tensors="pt")
    
    # デバイスに転送
    input_ids = encoded_inputs["input_ids"].to(device)
    attention_mask = encoded_inputs["attention_mask"].to(device)
    
    # モデルに入力
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # トークンごとの埋め込みを取得
    token_embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)
    
    return token_embeddings, attention_mask

longest_smiles_embedding_len = 0
longest_smiles_embedding_id = ''
longest_smiles = ''

with open(smiles_csv_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in tqdm(reader, desc='reading csv'):
        token_embedding, _ = smiles_to_token_embeddings([row['Canonical_SMILES']])
        if token_embedding.shape[1] > longest_smiles_embedding_len:
            longest_smiles_embedding_len = token_embedding.shape[1]
            longest_smiles_embedding_id = row['Ligand_id']
            longest_smiles = row['Canonical_SMILES']
        
print(f'longest smiles embedding length: {longest_smiles_embedding_len}')
print(f'longest smiles embedding id: {longest_smiles_embedding_id}')
print(f'longest smiles: {longest_smiles}')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
reading csv: 563066it [12:38, 742.16it/s]

longest smiles embedding length: 93
longest smiles embedding id: ZINC08214804
longest smiles: CO[C@H]1C[C@@H](O[C@@H]([C@H](C)[C@@H](O[C@@H]2O[C@H](C)C[C@H]([NH+](C)C)[C@@H]2O)C(C)C)[C@@H](C)C(=O)O[C@H](C)[C@H](C)[C@H](O)[C@@H](C)C(=O)[C@]2(C)CO2)O[C@@H](C)[C@H]1O


count active smiles numbers per uniprot id

In [18]:
with open(os.path.join('..', path_config['data']['DUD-E'], 'target_proteins.txt'), 'r') as f:
    target_proteins = f.read().splitlines()

ligand_counts = []

for target in target_proteins:
    with open(os.path.join(dude_dir, target, 'uniprot.txt'), 'r') as f:
        uniprot_ids = f.read().strip().splitlines()

    for uniprot in uniprot_ids:
        with open(os.path.join(dude_dir, target, uniprot + '.ism'), 'r') as f:
            num_ligands = sum(1 for line in f if line.strip())
        
        ligand_counts.append({'target': target, 'uniprot': uniprot, 'num_ligands': num_ligands})
        print(f'{target} {uniprot} {num_ligands}')

df = pd.DataFrame(ligand_counts)
df.to_csv(os.path.join(output_dir, 'ligand_counts.csv'), index=False)


aa2ar P29274 1948
aa2ar P30543 1376
aa2ar P46616 15
abl1 P00519 404
abl1 P00520 7
ace P09470 23
ace P12821 514
ace P12822 89
ace P47820 207
aces O42275 334
aces P04058 98
aces P21836 111
aces P22303 979
aces P23795 160
aces P37136 374
ada P00813 35
ada P03958 0
ada P56658 74
ada17 O77636 334
ada17 P78536 1111
ada17 Q9Z1K9 7
adrb1 P07700 0
adrb1 P08588 559
adrb1 P18090 108
adrb1 P34971 3
adrb1 Q9TT96 3
adrb2 P04274 4
adrb2 P07550 493
adrb2 P10608 21
adrb2 P18762 6
adrb2 P54833 75
adrb2 Q28044 37
adrb2 Q8K4Z4 22
akt1 P31749 587
akt1 P31750 1
akt2 P31751 237
aldr P07943 125
aldr P15121 372
aldr P16116 74
aldr P80276 59
ampc P00811 34
ampc P05364 66
ampc P24735 19
andr P10275 919
andr P15207 162
andr P19091 46
aofb P19643 255
aofb P27338 172
aofb P56560 50
aofb Q8BW75 1
bace1 P56817 600
braf P15056 317
cah2 P00918 1957
cah2 P00921 28
cah2 P27139 2
casp3 P42574 472
cdk2 P24941 1326
comt P21964 14
comt P22734 27
cp2c9 P11712 145
cp3a4 P08684 303
csf1r P07333 389
csf1r P09581 27
cxcr4 O08565 

In [20]:
interaction_file = os.path.join(dude_dir, 'pur2', 'glide-dock_merged_best_pv.interaction')

data = pd.read_csv(interaction_file, sep=',', header=None, comment='#')

data.iloc[:, [0, -1]]

,0,158
0,CHEMBL257691,-13.674135
1,CHEMBL23278,-13.429955
2,CHEMBL66389,-13.407990
3,CHEMBL315076,-13.234290
4,CHEMBL84163,-13.204518
...,...,...
2710,ZINC12877080,-2.460747
2711,ZINC34379976,-2.306156
2712,ZINC39560049,-2.224630
2713,ZINC43743762,-2.111117


analize preprocessed ligand dataset

In [30]:
smiles_csv_path = os.path.join(preprocessed_dir, 'dataset_final.csv')
print(smiles_csv_path)
longest_smiles = ''
longest_smiles_len = 0
longest_smiles_id = ''
total_count = 0

with open(smiles_csv_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        total_count += 1
        if len(row['Canonical_SMILES']) > len(longest_smiles):
            longest_smiles = row['Canonical_SMILES']
            longest_smiles_len = len(row['Canonical_SMILES'])
            longest_smiles_id = row['Ligand_id']

print(f'total: {total_count}')
print(f'longest smiles: {longest_smiles}')
print(f'longest smiles length: {longest_smiles_len}')
print(f'longest smiles id: {longest_smiles_id}')

.././data/preprocessed/dataset_final.csv
total: 563066
longest smiles: CO[C@H]1C[C@@H](O[C@@H]([C@H](C)[C@@H](O[C@@H]2O[C@@H](C)C[C@@H]([NH+](C)C)[C@@H]2O)C(C)C)[C@@H](C)C(=O)O[C@H](C)[C@H](C)[C@H](O)[C@@H](C)C(=O)[C@]2(C)CO2)O[C@@H](C)[C@H]1O
longest smiles length: 172
longest smiles id: ZINC08214807


analize protein representation

In [38]:
df = pd.read_csv(os.path.join(smiles_dir, 'available_uniprot.csv'))
available_uniprot = df['UniProt ID'].tolist()
protein_files = [os.path.join(alphafold_dir, uniprot, 'structure.npy') for uniprot in available_uniprot]

print(f"structure.npy shape: {np.load(protein_files[0]).shape[0]}")

protein_max_length = 0
for file in protein_files:
    if np.load(file).shape[0] > protein_max_length:
        protein_max_length = np.load(file).shape[0]

print(f"max protein length: {protein_max_length}")


structure.npy shape: 412
max protein length: 1390
